In [1]:
import numpy as np
import pickle
import keras.backend as K

from keras.layers import  GlobalAveragePooling1D, Input, Activation, MaxPooling1D, BatchNormalization, Dense, Dropout, Conv1D,GlobalMaxPooling1D
from keras.layers import GRU,AveragePooling1D,CuDNNGRU
from keras.layers.merge import Concatenate
from keras.models import Model 
from keras.callbacks import EarlyStopping,ModelCheckpoint


import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
import os


os.environ["CUDA_VISIBLE_DEVICES"] = "1"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

KTF.set_session(sess)

Using TensorFlow backend.
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xhh/anaconda3/envs/py36/lib/p

In [2]:
import numpy as np
alphabet = np.array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
                     'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])

def label_sequence(line, MAX_SEQ_LEN, smi_ch_ind):
	X = np.zeros(MAX_SEQ_LEN)

	for i, ch in enumerate(line[:MAX_SEQ_LEN]):
		X[i] = smi_ch_ind[ch]

	return X #.tolist()

def letter_one_hot(aa):
    one_hot = np.zeros(20)
    for idx, letter in enumerate(alphabet):
        if aa == letter:
            one_hot[idx] = 1
            return one_hot


# Convert an entire protein to one-hot representation.
def protein_one_hot(protein_sequence, MAX_SEQ_LEN):
    #  Remove non-specific AA codes (very few are actually present in this dataset)
    protein_sequence = protein_sequence.replace('B', '')
    protein_sequence = protein_sequence.replace('J', '')
    protein_sequence = protein_sequence.replace('O', '')
    protein_sequence = protein_sequence.replace('U', '')
    protein_sequence = protein_sequence.replace('X', '')
    protein_sequence = protein_sequence.replace('Z', '')
    one_hot_seq = np.zeros( (MAX_SEQ_LEN, 20))
    for idx, aa in enumerate(protein_sequence[:MAX_SEQ_LEN]):
        one_hot_seq[idx, :] = letter_one_hot(aa)
    return one_hot_seq




In [3]:
import keras
feature_len = 768
max_go_len = 128
max_seq_len = 1000

from six.moves import cPickle as pickle #for performance

 
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di



class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,  ppi_pair_file, batch_size=128):
        'Initialization' 
        self.batch_size = batch_size
        self.ppi_pair_file = ppi_pair_file
         
        self.max_seqlen = max_seq_len
        self.max_golen = max_go_len
        
        self.protein2seq = load_dict('yeast_data/protein2seq_dicts.pkl')
        self.read_ppi()
         
        self.protein2onehot = {}
        self.onehot_seqs()
        self.on_epoch_end()
    
    def read_ppi(self):
        with open(self.ppi_pair_file, 'r') as f:
            self.ppi_pairs  =  f.readlines()
            
    
    def onehot_seqs(self):
        for key, value in self.protein2seq.items():
            self.protein2onehot[key] =  protein_one_hot(value, self.max_seqlen) 
#             self.protein2onehot[key]  = seq2t.embed_normalized(value,self.max_seqlen )

    

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.ppi_pairs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y
 
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.ppi_pairs))
         
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

        
        y = np.empty((self.batch_size))
        X_seq1 = np.empty((self.batch_size, self.max_seqlen,20))
        X_seq2 = np.empty((self.batch_size, self.max_seqlen,20))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ppi_pair = self.ppi_pairs[ID]
            p1, p2, label = ppi_pair.rstrip().split('\t')
            if label == '1':
                y[i] = 1
            else:
                y[i] = 0
                
            
            
            X_seq1[i] =  self.protein2onehot[p1]
            X_seq2[i] =  self.protein2onehot[p2]
     
        return [  X_seq1, X_seq2] ,  y



    def all_data(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

        
        y = np.empty((len(list_IDs_temp)))
        
        X_seq1 = np.empty((len(list_IDs_temp), self.max_seqlen,20))
        X_seq2 = np.empty((len(list_IDs_temp), self.max_seqlen,20))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ppi_pair = self.ppi_pairs[ID]
            p1, p2, label = ppi_pair.rstrip().split('\t')
            if label == '1':
                y[i] = 1
            else:
                y[i] = 0
             
            X_seq1[i] =  self.protein2onehot[p1]
            X_seq2[i] =  self.protein2onehot[p2]
            
  
        return [X_seq1, X_seq2] ,  y

In [4]:
from keras import backend as K, initializers, regularizers, constraints
from keras.engine.topology import Layer


def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        # todo: check that this is correct
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True,
                 return_attention=False,
                 **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Note: The layer has been tested with Keras 1.x
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2 - Get the attention scores
            hidden = LSTM(64, return_sequences=True)(words)
            sentence, word_scores = Attention(return_attention=True)(hidden)
        """
        self.supports_masking = True
        self.return_attention = return_attention
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        eij = dot_product(x, self.W)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        weighted_input = x * K.expand_dims(a)

        result = K.sum(weighted_input, axis=1)

        if self.return_attention:
            return [result, a]
        return result

    def compute_output_shape(self, input_shape):
        if self.return_attention:
            return [(input_shape[0], input_shape[-1]),
                    (input_shape[0], input_shape[1])]
        else:
            return input_shape[0], input_shape[-1]

In [6]:
from keras.layers import   Embedding
from keras.layers import  GRU, Bidirectional, CuDNNGRU, Lambda, dot, Flatten, CuDNNLSTM, Add
from keras.layers.merge import concatenate
from keras_radam import RAdam
from keras_lookahead import Lookahead
def inception_block(input_tensor, output_size):
    """"""
    con1d_filters = int(output_size/4)
    y = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x1 = Conv1D(con1d_filters, 5, activation="relu", padding='same')(y)

    y = Conv1D(con1d_filters, 1, activation="relu", padding='valid')(input_tensor)
    x2 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(y)

    x3 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x4 = Conv1D(con1d_filters, 1, activation="relu", padding='same')(input_tensor)

    y = Concatenate()([x1, x2, x3, x4])
#     y = MaxPooling1D(2)(mix0)
    # y = AveragePooling1D()(mix0)
#     y = BatchNormalization()(y)

    return y


 


def build_cnn_gru_model(input_x, con_filters, gru_units):
    x = inception_block(input_x,con_filters )
    x = Dropout(0.3)(x)
    x_gru = Bidirectional(CuDNNGRU(gru_units, return_sequences=True))(input_x)
    x_gru = Dropout(0.3)(x_gru)
     
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_c = Attention()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)
    x_gru_c = Attention()(x_gru)
    x = Concatenate()([x_a, x_b, x_c, x_gru_a, x_gru_b,   x_gru_c])
    x = Dense(256,activation='relu')(x)
    return x


def build_cnn_model(input_x, con_filters, gru_units):
    x = inception_block(input_x,con_filters )
    x = Dropout(0.3)(x)
    
     
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_c = Attention()(x)
    
    x = Concatenate()([ x_a, x_b, x_c])
    x = Dense(256,activation='relu')(x)
    return x 


def build_model():
    con_filters = 256
    gru_units = 64
     
    
     
    
    
    left_input_seq = Input(shape=(max_seq_len,20))
    right_input_seq = Input(shape=(max_seq_len,20))
    

      
    left_x_seq = build_cnn_gru_model(left_input_seq, con_filters//4, gru_units)
    right_x_seq = build_cnn_gru_model(right_input_seq, con_filters//4, gru_units)
     
   
    x =   Concatenate()([ left_x_seq, right_x_seq])
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(1)(x)
#     x = BatchNormalization()(x)
    output = Activation('sigmoid')(x)
    # model = Model([left_input_go, right_input_go], output)
  
    model = Model([ left_input_seq, right_input_seq], output)
#     model = multi_gpu_model(model)
#     rms = RMSprop(lr=0.0001)
#     adam = Adam(lr=0.001, amsgrad=True, epsilon=1e-5)
    
    optimizer = Lookahead(RAdam())

    model.compile(loss='binary_crossentropy', optimizer= optimizer, metrics=['accuracy'])
    return model


 

model = build_model()
model.summary()
 



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1000, 20)     0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1000, 20)     0                                            
__________________________________________________________________________________________________
conv1d_13 (Conv1D)              (None, 1000, 16)     976         input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_15 (Conv1D)              (None, 1000, 16)     336         input_3[0][0]                 

In [9]:
from sklearn.model_selection import StratifiedKFold
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
dataset_name = 'SC'
for rep in range(1):
    n_splits = 5
    TPRs =  np.zeros(n_splits)
    FPRs = np.zeros(n_splits)
    Precs = np.zeros(n_splits)
    ACCs = np.zeros(n_splits)
    F1s = np.zeros(n_splits)
    MCCs = np.zeros(n_splits)
    AUCs = np.zeros(n_splits)
    skf = StratifiedKFold(n_splits=n_splits)
    count = 0
    for split in range(n_splits):
        train_pairs_file = 'yeast_data/new_train_valid-'+str(split) 
        test_pairs_file = 'yeast_data/new_test-'+str(split) 
#         valid_pairs_file = 'yeast_data/valid'+str(rep)+'-'+str(split)

        batch_size = 256
        train_generator = DataGenerator(   train_pairs_file,batch_size = batch_size )
        test_generator = DataGenerator(   test_pairs_file,batch_size = batch_size)
#         valid_generator = DataGenerator(   valid_pairs_file,batch_size = batch_size)
         
        # model = build_model_without_att()
        model = build_model()
        save_model_name = 'yeast_data/sc_seq'+str(rep)+'-'+str(split) + '.hdf5'
        
        earlyStopping = EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='min')
        save_checkpoint = ModelCheckpoint(save_model_name, save_best_only=True, monitor='val_loss', mode='min', save_weights_only=True)

         
        # validation_data = (valid_X, valid_Y),  verbose=1,callbacks=[earlyStopping, save_checkpoint]
        hist = model.fit_generator(generator=train_generator,
                    epochs = 100,verbose=1)
         
        
        # model = load_model(save_model_name)
#         model.load_weights(save_model_name)
        with open(test_pairs_file, 'r') as f:
            test_ppi_pairs  =  f.readlines()

        test_len = len(test_ppi_pairs) 
        list_IDs_temp = np.arange(test_len)

        test_x, y_test = test_generator.all_data(list_IDs_temp)

        y_pred_prob = model.predict(test_x)

       
        y_pred = (y_pred_prob > 0.5)
        auc = metrics.roc_auc_score(y_test, y_pred_prob) 
        f1 = f1_score(y_test, y_pred)
        pre = precision_score(y_test, y_pred)
        acc = accuracy_score(y_test, y_pred)

        precision, recall, _thresholds = metrics.precision_recall_curve(y_test, y_pred_prob)
        pr_auc = metrics.auc(recall, precision)
        mcc = matthews_corrcoef(y_test, y_pred)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total=tn+fp+fn+tp
        sen = float(tp)/float(tp+fn)
        sps = float(tn)/float((tn+fp))

        tpr = float(tp)/float(tp+fn)
        fpr = float(fp)/float((tn+fp))
        print('--------------------------\n')
        print ('AUC: %f' % auc)
        print ('ACC: %f' % acc) 
        # print("PRAUC: %f" % pr_auc)
        print ('MCC : %f' % mcc)
        # print ('SEN: %f' % sen)
        # print ('SEP: %f' % sps)
        print('TPR:%f'%tpr)
        print('FPR:%f'%fpr)
        print('Pre:%f'%pre)
        print('F1:%f'%f1)
        print('--------------------------\n')
        TPRs[count] = tpr
        FPRs[count] = fpr
        Precs[count] =pre
        ACCs[count] =acc
        F1s[count] =f1
        MCCs[count] =mcc
        AUCs[count] =auc
        count += 1
        del test_x
        del y_test
    print ('mean AUC: %f' % np.mean(AUCs))
    print ('mean ACC: %f' % np.mean(ACCs)) 
    print ('mean MCC : %f' % np.mean(MCCs))
    print('mean TPR:%f'% np.mean(TPRs))
    print('mean FPR:%f'% np.mean(FPRs))
    print('mean Pre:%f'% np.mean(Precs))
    print('mean F1:%f'% np.mean(F1s))
    np.savez('yeast_seq'+str(rep), AUCs=AUCs, ACCs=ACCs, MCCs=MCCs, TPRs = TPRs, FPRs=FPRs, Precs=Precs, F1s=F1s)

Epoch 1/100
34/34 [==============================] - 14s 401ms/step - loss: 0.6662 - acc: 0.6202
Epoch 2/100
34/34 [==============================] - 8s 241ms/step - loss: 0.6613 - acc: 0.6253
Epoch 3/100
34/34 [==============================] - 8s 245ms/step - loss: 0.6619 - acc: 0.6253
Epoch 4/100
34/34 [==============================] - 8s 246ms/step - loss: 0.6604 - acc: 0.6253
Epoch 5/100
34/34 [==============================] - 8s 243ms/step - loss: 0.6617 - acc: 0.6253
Epoch 6/100
34/34 [==============================] - 8s 245ms/step - loss: 0.6603 - acc: 0.6253
Epoch 7/100
34/34 [==============================] - 8s 246ms/step - loss: 0.6590 - acc: 0.6253
Epoch 8/100
34/34 [==============================] - 8s 247ms/step - loss: 0.6607 - acc: 0.6255
Epoch 9/100
34/34 [==============================] - 8s 244ms/step - loss: 0.6593 - acc: 0.6253
Epoch 10/100
34/34 [==============================] - 8s 246ms/step - loss: 0.6573 - acc: 0.6253
Epoch 11/100
34/34 [==================

34/34 [==============================] - 8s 243ms/step - loss: 0.1776 - acc: 0.9273
Epoch 86/100
34/34 [==============================] - 8s 243ms/step - loss: 0.1877 - acc: 0.9276
Epoch 87/100
34/34 [==============================] - 8s 242ms/step - loss: 0.1684 - acc: 0.9323
Epoch 88/100
34/34 [==============================] - 8s 246ms/step - loss: 0.1661 - acc: 0.9332
Epoch 89/100
34/34 [==============================] - 8s 245ms/step - loss: 0.1929 - acc: 0.9220
Epoch 90/100
34/34 [==============================] - 8s 245ms/step - loss: 0.1610 - acc: 0.9361
Epoch 91/100
34/34 [==============================] - 8s 246ms/step - loss: 0.1793 - acc: 0.9260
Epoch 92/100
34/34 [==============================] - 8s 246ms/step - loss: 0.1448 - acc: 0.9427
Epoch 93/100
34/34 [==============================] - 8s 246ms/step - loss: 0.1763 - acc: 0.9293
Epoch 94/100
34/34 [==============================] - 8s 244ms/step - loss: 0.2255 - acc: 0.9084
Epoch 95/100
34/34 [=======================

34/34 [==============================] - 8s 247ms/step - loss: 0.2645 - acc: 0.8857
Epoch 69/100
34/34 [==============================] - 8s 248ms/step - loss: 0.2878 - acc: 0.8783
Epoch 70/100
34/34 [==============================] - 8s 246ms/step - loss: 0.3080 - acc: 0.8655
Epoch 71/100
34/34 [==============================] - 8s 246ms/step - loss: 0.2733 - acc: 0.8851
Epoch 72/100
34/34 [==============================] - 8s 248ms/step - loss: 0.2521 - acc: 0.8988
Epoch 73/100
34/34 [==============================] - 8s 246ms/step - loss: 0.2454 - acc: 0.8965
Epoch 74/100
34/34 [==============================] - 9s 250ms/step - loss: 0.2648 - acc: 0.8880
Epoch 75/100
34/34 [==============================] - 8s 246ms/step - loss: 0.2524 - acc: 0.8937
Epoch 76/100
34/34 [==============================] - 8s 245ms/step - loss: 0.2572 - acc: 0.8940
Epoch 77/100
34/34 [==============================] - 8s 247ms/step - loss: 0.2627 - acc: 0.8867
Epoch 78/100
34/34 [=======================

34/34 [==============================] - 8s 247ms/step - loss: 0.4155 - acc: 0.8174
Epoch 52/100
34/34 [==============================] - 9s 250ms/step - loss: 0.3606 - acc: 0.8458
Epoch 53/100
34/34 [==============================] - 8s 249ms/step - loss: 0.3543 - acc: 0.8469
Epoch 54/100
34/34 [==============================] - 8s 250ms/step - loss: 0.3737 - acc: 0.8332
Epoch 55/100
34/34 [==============================] - 8s 248ms/step - loss: 0.3794 - acc: 0.8286
Epoch 56/100
34/34 [==============================] - 8s 247ms/step - loss: 0.3262 - acc: 0.8617
Epoch 57/100
34/34 [==============================] - 8s 247ms/step - loss: 0.3313 - acc: 0.8570
Epoch 58/100
34/34 [==============================] - 8s 247ms/step - loss: 0.3281 - acc: 0.8559
Epoch 59/100
34/34 [==============================] - 9s 250ms/step - loss: 0.3374 - acc: 0.8524
Epoch 60/100
34/34 [==============================] - 8s 249ms/step - loss: 0.3333 - acc: 0.8523
Epoch 61/100
34/34 [=======================

34/34 [==============================] - 9s 253ms/step - loss: 0.4370 - acc: 0.7981
Epoch 35/100
34/34 [==============================] - 9s 256ms/step - loss: 0.4508 - acc: 0.7869
Epoch 36/100
34/34 [==============================] - 9s 258ms/step - loss: 0.4603 - acc: 0.7838
Epoch 37/100
34/34 [==============================] - 9s 257ms/step - loss: 0.4695 - acc: 0.7807
Epoch 38/100
34/34 [==============================] - 9s 254ms/step - loss: 0.4241 - acc: 0.8097
Epoch 39/100
34/34 [==============================] - 9s 256ms/step - loss: 0.5045 - acc: 0.7502
Epoch 40/100
34/34 [==============================] - 9s 253ms/step - loss: 0.4287 - acc: 0.8030
Epoch 41/100
34/34 [==============================] - 9s 253ms/step - loss: 0.3930 - acc: 0.8238
Epoch 42/100
34/34 [==============================] - 9s 256ms/step - loss: 0.4079 - acc: 0.8228
Epoch 43/100
34/34 [==============================] - 9s 255ms/step - loss: 0.3939 - acc: 0.8253
Epoch 44/100
34/34 [=======================

34/34 [==============================] - 9s 252ms/step - loss: 0.6003 - acc: 0.6870
Epoch 18/100
34/34 [==============================] - 8s 249ms/step - loss: 0.5922 - acc: 0.6842
Epoch 19/100
34/34 [==============================] - 8s 248ms/step - loss: 0.5632 - acc: 0.7139
Epoch 20/100
34/34 [==============================] - 9s 250ms/step - loss: 0.5634 - acc: 0.7122
Epoch 21/100
34/34 [==============================] - 9s 253ms/step - loss: 0.5449 - acc: 0.7277
Epoch 22/100
34/34 [==============================] - 9s 252ms/step - loss: 0.5437 - acc: 0.7267
Epoch 23/100
34/34 [==============================] - 9s 253ms/step - loss: 0.5372 - acc: 0.7344
Epoch 24/100
34/34 [==============================] - 9s 251ms/step - loss: 0.5335 - acc: 0.7335
Epoch 25/100
34/34 [==============================] - 8s 249ms/step - loss: 0.5089 - acc: 0.7554
Epoch 26/100
34/34 [==============================] - 9s 252ms/step - loss: 0.4946 - acc: 0.7645
Epoch 27/100
34/34 [=======================